<a href="https://colab.research.google.com/github/pablogadhi/TranslationDetector/blob/master/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pablogadhi/TranslationDetector
!python -m spacy download en
!python -m spacy download es
%cd TranslationDetector
!pip install --upgrade torch torchtext

Cloning into 'TranslationDetector'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 36 (delta 11), reused 29 (delta 8), pack-reused 0
Unpacking objects: 100% (36/36), done.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 16.2MB 1.2MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172936 sha256=4954139668a5329c5cc0d8482088ead09d6da37966bfbde796274e5d0f7470c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-w41xbpd0/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load t

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [22]:
!git pull

Already up to date.


In [0]:
import torch
from translation.train import train_model
from translation.data_loader import load_data, make_iters
from translation.transformer import Transformer

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [10]:
!nvidia-smi

Wed Jun 10 06:25:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    30W / 149W |     11MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Load data and create the model

In [0]:
DATA_DIR='/content/drive/My Drive/NLP/data/en-es_'

In [0]:
SRC, TGT, train, valid, test = load_data(
        DATA_DIR, "en.txt", "es.txt")

In [0]:
train_itr, valid_itr, test_itr = make_iters(
        train, valid, test, device, batch_size=3000)
model = Transformer(len(SRC.vocab), len(TGT.vocab)).to(device)

In [16]:
len(SRC.vocab.itos)

40002

### Train the model

In [24]:
train_model(model, train_itr, valid_itr, SRC, TGT, device, 10, save_at=1)

NameError: ignored